In [1]:
%config IPCompleter.greedy=True

In [2]:
import plotly
import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, clear_output
import ipywidgets as widgets
import pandas as pd

In [3]:
cData = pd.read_csv('COVID-19-Data/aggregated_sorted_totall.csv')

In [4]:
# Main GUI
# Create Figures Buttons
title_label  = widgets.Label(value='Choose Visualization Mode')
bubble_btn   = widgets.Button(description='Bubble')
maps_btn     = widgets.Button(description='Maps')
bar_btn      = widgets.Button(description='Bar')
btns_container = widgets.VBox([title_label, widgets.HBox([bubble_btn, maps_btn, bar_btn])])
outputFigure   = widgets.Output()
bubbleFigure = px.scatter(cData, x="Deaths",
                                 y="Recovered",
                                 size="Confirmed",
                                 color="Country",
                                 animation_frame="Date",
                                 animation_group="Country",
                                 hover_name="Country",
                                 log_x=True,
#                                  size_max=200,
#                                  range_x=[1, 85000],
#                                  range_y=[0, 250000]
                         )

mapsFigure = px.choropleth(cData, locations='Country',
                                  locationmode='country names',
                                  color='Confirmed',
                                  color_continuous_scale=px.colors.sequential.Plasma,
#                                   range_color=(0, 100),
                                  labels={'Country':'Confirmed'},
                                  hover_name='Country',
                                  animation_frame="Date",
                                  animation_group="Country")

barFigure = px.bar(cData, x="Country",
                          y="Confirmed",
                          color='Country',
                          animation_frame="Date",
                          animation_group="Country")

def generateBubbleGraph(_):
    with outputFigure:
        clear_output()
        bubbleFigure.show()

def generateMapsGraph(_):
    with outputFigure:
        clear_output()
        mapsFigure.show()
        
def generateBarGraph(_):
    with outputFigure:
        clear_output()
        barFigure.show()

In [6]:
# Buttons Handler 
bubble_btn.on_click(generateBubbleGraph)
maps_btn.on_click(generateMapsGraph)
bar_btn.on_click(generateBarGraph)
widgets.VBox([btns_container, outputFigure])

In [11]:
# modeButtons = widgets.ToggleButtons(
#     options=['Bubble', 'Maps', 'Bar'],
# #     description='View Mode:',
#     disabled=False,
#     button_style='info', # 'success', 'info', 'warning', 'danger' or ''
#     tooltips=['Bubble Graph', 'Maps Graph', 'Bar Rank'],
# )
# modeButtons.observe(SelectDisplayMode, names='value')

In [6]:
# def SelectDisplayMode(_):
#     with outputFigure:
#         clear_output()
#         print('Hellooo')
#         # Display Bubble Graph
#         if (modeButtons.value == 'Bubble'):
#             print(f'Mode: {modeButtons.value}')
#             fig = px.scatter(cData, x="Deaths", y="Recovered", animation_frame="Date", animation_group="Country",
#                        size="Confirmed", color="Country", hover_name="Country",
#                        log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90])
#             fig.show()
#             plotly.offline.iplot(fig)
#
#         # Display Maps Graph
#         if (modeButtons.value == 'Maps'):
#             print(f'Mode: {modeButtons.value}')
#             #initializing the data variable
#             data = dict(type = 'choropleth',
#                         locations = ['india','nepal','china','pakistan','Bangladesh','bhutan','myanmar','srilanka','egypt', 'turkey'],
#                         locationmode = 'country names',
#                         colorscale= 'Portland',
#                         text= ['IND','NEP','CHI','PAK','BAN','BHU', 'MYN','SLK', 'EGY', 'TUR'],
#                         z=[1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0],
#                         colorbar = {'title':'Country Colours', 'len':200,'lenmode':'pixels' })
#             col_map = go.Figure(data = [data])
#             g = go.FigureWidget(col_map)
#             g.show()
#
#         # Display Bar Rank Graph
#         if (modeButtons.value == 'Bar'):
#             print(f'Mode: {modeButtons.value}')
#             fig = px.bar(cData, x="Country", y="Confirmed", color='Country',
#                          animation_frame="Date", animation_group="Country")
#             fig.show()

In [ ]:
# def generateBubbleGraph(_):
#     with outputFigure:
#         clear_output()
#         bubbleFigure.show()

# #         fig = px.scatter(cData, x="Deaths",
# #                                 y="Recovered",
# #                                 size="Confirmed",
# #                                 color="Country",
# #                                 animation_frame="Date",
# #                                 animation_group="Country",
# #                                 hover_name="Country",
# #                                 size_max=30000)
# #         fig.show()

# def generateMapsGraph(_):
#     with outputFigure:
#         clear_output()
#         mapsFigure.show()
# #         fig = px.choropleth(cData, locations='Country',
# #                                    locationmode='country names',
# #                                    color='Confirmed',
# #                                    color_continuous_scale=px.colors.sequential.Plasma,
# #                                    labels={'Country':'Confirmed'},
# #                                    hover_name='Country',
# #                                    animation_frame="Date",
# #                                    animation_group="Country")
# #         fig.show()